In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
pariza_bbc_news_summary_path = kagglehub.dataset_download('pariza/bbc-news-summary')
gauravduttakiit_resume_dataset_path = kagglehub.dataset_download('gauravduttakiit/resume-dataset')
nileshmalode1_samsum_dataset_text_summarization_path = kagglehub.dataset_download('nileshmalode1/samsum-dataset-text-summarization')
lukebarousse_data_analyst_job_postings_google_search_path = kagglehub.dataset_download('lukebarousse/data-analyst-job-postings-google-search')

print('Data source import complete.')


100%|██████████| 8.91M/8.91M [00:01<00:00, 8.36MB/s]


Extracting files...


100%|██████████| 383k/383k [00:00<00:00, 739kB/s]

Extracting files...


100%|██████████| 7.99M/7.99M [00:01<00:00, 7.62MB/s]

Extracting files...


100%|██████████| 80.5M/80.5M [00:04<00:00, 17.7MB/s]

Extracting files...


Data source import complete.


In [ ]:
gauravduttakiit_resume_dataset_path

'/root/.cache/kagglehub/datasets/gauravduttakiit/resume-dataset/versions/1'

# Transformer-based Approach for Semantic Job Description & Resume Matching

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

> **The individual task is devoted to data analysis and model verification (literature review, visits to laboratories, etc. are not relevant at this stage of your thesis!)**

# 1. Preprocessing

1. Cleaning Text Data
2. Lowercasing
3. Tokenization
4. Removing Stop Words
5. Stemming/Lemmatization
6. Handling Missing Data

In [ ]:
# Libraries

import nltk
import zipfile
import os
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

**Jobs Data**

In [ ]:
# Read jobs data

import pandas as pd

jobs_data = pd.read_csv(lukebarousse_data_analyst_job_postings_google_search_path + '/gsearch_jobs.csv')

In [ ]:
jobs_data

,Unnamed: 0,index,title,company_name,location,via,description,extensions,job_id,thumbnail,...,commute_time,salary_pay,salary_rate,salary_avg,salary_min,salary_max,salary_hourly,salary_yearly,salary_standardized,description_tokens
0,0,0,Data Analyst,Chloeta,"Oklahoma City, OK",via ZipRecruiter,Job Summary: The Data Analyst oversees data pr...,"['21 hours ago', 'Full-time', 'Health insuranc...",eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QiLCJodGlkb2...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['r', 'python']"
1,1,1,Junior Data Analyst/Scientist Role - Contract ...,Upwork,Anywhere,via Upwork,"Company\n\nThe TAC Index provides independent,...","['17 hours ago', 'Work from home', 'Contractor...",eyJqb2JfdGl0bGUiOiJKdW5pb3IgRGF0YSBBbmFseXN0L1...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['python', 'r', 'mysql', 'matplotlib', 'matlab..."
2,2,2,Data Analyst,ATC,United States,via LinkedIn,Job Title: Entry Level Business Analyst / Prod...,"['12 hours ago', 'Full-time', 'Health insurance']",eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QiLCJodGlkb2...,https://encrypted-tbn0.gstatic.com/images?q=tb...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
3,3,3,Data Analyst,Guidehouse,"Topeka, KS",via Nexxt,Job Family :\n\nData Science & Analysis (Digit...,"['10 hours ago', 'Full-time', 'Health insuranc...",eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QiLCJodGlkb2...,https://encrypted-tbn0.gstatic.com/images?q=tb...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['power_bi', 'powerpoint', 'excel', 'word', 'r..."
4,4,4,Data Analyst,AnMed Health LLC,Anywhere,via LinkedIn,"AnMed is a dynamic, comprehensive health syste...","['18 hours ago', 'Work from home', 'Part-time'...",eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QiLCJodGlkb2...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59637,59637,1316,"Data Analyst, Supply Chain",The Home Depot,United States,via NRF Job Board - National Retail Federation,Position Purpose:\n\nA Data Analyst Supply Cha...,"['14 hours ago', 'Full-time']",eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QsIFN1cHBseS...,https://encrypted-tbn0.gstatic.com/images?q=tb...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['sql']
59638,59638,1317,Planning Analyst,Microsoft,United States,via Jobs | The FutureList Job Board - The Futu...,The Cloud Operations + Innovation (CO+I) organ...,"['9 hours ago', 'Full-time', 'Health insurance']",eyJqb2JfdGl0bGUiOiJQbGFubmluZyBBbmFseXN0IiwiaH...,https://encrypted-tbn0.gstatic.com/images?q=tb...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['excel', 'sql']"
59639,59639,1318,"Data Analyst, Business Services",Community College of Baltimore County,United States,via Higher Education Recruitment Consortium (H...,The purpose of this class is to perform profes...,"['20 hours ago', 'Full-time']",eyJ1dWxlIjoidytDQUlRSUNJTlZXNXBkR1ZrSUZOMFlYUm...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['sql', 'power_bi', 'excel']"
59640,59640,1319,Data Analyst 2,ORAU,Anywhere,via Oak Ridge Associated Universities | Career...,Overview\n\nAppointment Type: Full-Time Regula...,"['20 hours ago', 'Work from home', 'Full-time'...",eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QgMiIsImh0aW...,https://encrypted-tbn0.gstatic.com/images?q=tb...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['spreadsheet', 'excel', 'power_bi', 'r', 'sql..."


In [ ]:
jobs_data['description']

,description
0,Job Summary: The Data Analyst oversees data pr...
1,"Company\n\nThe TAC Index provides independent,..."
2,Job Title: Entry Level Business Analyst / Prod...
3,Job Family :\n\nData Science & Analysis (Digit...
4,"AnMed is a dynamic, comprehensive health syste..."
...,...
59637,Position Purpose:\n\nA Data Analyst Supply Cha...
59638,The Cloud Operations + Innovation (CO+I) organ...
59639,The purpose of this class is to perform profes...
59640,Overview\n\nAppointment Type: Full-Time Regula...


In [ ]:
!pip install --upgrade scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 11.9 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.15.2 which is incompatible.


In [ ]:
import re
import zipfile
import os
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [ ]:

import nltk

def setup_nltk_resources():
    """
    Sets up the custom NLTK data path and downloads necessary resources.
    Downloads 'wordnet' for lemmatization, 'stopwords' for stopwords removal,
    and 'punkt' for sentence tokenization.
    """
    nltk_data_path = "/root/nltk_data"
    nltk.data.path.append(nltk_data_path)

    nltk.download('wordnet', download_dir=nltk_data_path)
    nltk.download('stopwords', download_dir=nltk_data_path)
    nltk.download('punkt', download_dir=nltk_data_path)
    nltk.download('punkt_tab', download_dir=nltk_data_path) # Download the missing resource

In [ ]:
def unzip_nltk_resource(zip_path, extract_to):
    """
    Unzips an NLTK resource file to a specified directory.

    Args:
    zip_path (str): The path to the zipped NLTK resource file.
    extract_to (str): The directory where the contents of the zip file will be extracted.
    """
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

In [ ]:
def preprocess_text(text):
    """
    Preprocesses a given text string for NLP tasks. This includes cleaning the text,
    tokenizing, removing stopwords, and lemmatizing the words.

    Args:
    text (str): The text string to preprocess.

    Returns:
    str: The preprocessed text.
    """
    if not text:
        return ""
    text = re.sub(r'[\r\n\t]+', ' ', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()

    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]

    lemmatizer = WordNetLemmatizer()
    lemmatized_text = [lemmatizer.lemmatize(word) for word in filtered_tokens]

    return ' '.join(lemmatized_text)

In [ ]:
# Setup NLTK resources
setup_nltk_resources()

# Unzip wordnet
corpora_path = "/root/nltk_data/corpora"
wordnet_zip = os.path.join(corpora_path, "wordnet.zip")
unzip_nltk_resource(wordnet_zip, corpora_path)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
# Apply preprocessing
jobs_data['processed_description'] = jobs_data['description'].apply(preprocess_text)
jobs_data['processed_title'] = jobs_data['title'].apply(preprocess_text)

In [ ]:
jobs_data['processed_description']

,processed_description
0,job summary data analyst oversees data product...
1,company tac index provides independent accurat...
2,job title entry level business analyst product...
3,job family data science analysis digital trave...
4,anmed dynamic comprehensive health system ande...
...,...
59637,position purpose data analyst supply chain lev...
59638,cloud operation innovation coi organization lo...
59639,purpose class perform professional level techn...
59640,overview appointment type fulltime regular ftr...


In [ ]:
jobs_data['processed_title']

,processed_title
0,data analyst
1,junior data analystscientist role contract hire
2,data analyst
3,data analyst
4,data analyst
...,...
59637,data analyst supply chain
59638,planning analyst
59639,data analyst business service
59640,data analyst


**Handle duplicates**

In [ ]:
def drop_duplicates(df, column_name):
    """
    Drops duplicates based on a specified column from the DataFrame.

    Args:
    df (pd.DataFrame): The DataFrame from which to remove duplicates.
    column_name (str): The name of the column based on which duplicates will be identified.

    Returns:
    pd.DataFrame: DataFrame with duplicates removed based on the specified column.
    """
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in DataFrame")

    original_size = df.shape[0]
    df_cleaned = df.drop_duplicates(subset=[column_name])
    new_size = df_cleaned.shape[0]

    print(f"Dropped {original_size - new_size} duplicates from '{column_name}'. New dataset size: {new_size}")

    return df_cleaned

In [ ]:
jobs_data_cleaned = drop_duplicates(jobs_data, column_name='description')

Dropped 18140 duplicates from 'description'. New dataset size: 41562


In [ ]:
jobs_data_cleaned['processed_description']

,processed_description
0,job summary data analyst oversees data product...
1,company tac index provides independent accurat...
2,job title entry level business analyst product...
3,job family data science analysis digital trave...
4,anmed dynamic comprehensive health system ande...
...,...
59637,position purpose data analyst supply chain lev...
59638,cloud operation innovation coi organization lo...
59639,purpose class perform professional level techn...
59640,overview appointment type fulltime regular ftr...


In [ ]:
jobs_data_cleaned['processed_title']

,processed_title
0,data analyst
1,junior data analystscientist role contract hire
2,data analyst
3,data analyst
4,data analyst
...,...
59637,data analyst supply chain
59638,planning analyst
59639,data analyst business service
59640,data analyst


**Resume data**

In [ ]:
resume_data = pd.read_csv( gauravduttakiit_resume_dataset_path + '/UpdatedResumeDataSet.csv')

In [ ]:
resume_data['Resume']

,Resume
0,Skills * Programming Languages: Python (pandas...
1,Education Details \r\nMay 2013 to May 2017 B.E...
2,"Areas of Interest Deep Learning, Control Syste..."
3,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,"Education Details \r\n MCA YMCAUST, Faridab..."
...,...
957,Computer Skills: â¢ Proficient in MS office (...
958,â Willingness to accept the challenges. â ...
959,"PERSONAL SKILLS â¢ Quick learner, â¢ Eagerne..."
960,COMPUTER SKILLS & SOFTWARE KNOWLEDGE MS-Power ...


In [ ]:
# Apply preprocessing
resume_data['processed_resume'] = resume_data['Resume'].apply(preprocess_text)

In [ ]:
resume_data['processed_resume']

,processed_resume
0,skill programming language python panda numpy ...
1,education detail may may uitrgpv data scientis...
2,area interest deep learning control system des...
3,skill r python sap hana tableau sap hana sql s...
4,education detail mca ymcaust faridabad haryana...
...,...
957,computer skill proficient m office word basic ...
958,willingness accept challenge positive thinking...
959,personal skill quick learner eagerness learn n...
960,computer skill software knowledge mspower poin...


In [ ]:
resume_data_cleaned = drop_duplicates(resume_data, column_name='Resume')

Dropped 796 duplicates from 'Resume'. New dataset size: 166


In [ ]:
resume_data_cleaned['processed_resume']

,processed_resume
0,skill programming language python panda numpy ...
1,education detail may may uitrgpv data scientis...
2,area interest deep learning control system des...
3,skill r python sap hana tableau sap hana sql s...
4,education detail mca ymcaust faridabad haryana...
...,...
894,computer skill proficient m office word basic ...
895,willingness accept challenge positive thinking...
896,personal skill quick learner eagerness learn n...
897,computer skill software knowledge mspower poin...


**Summary for Preprocessed Features for Resume and Job Descriptions**

1. resume_data_cleaned['processed_resume']
2. jobs_data_cleaned['processed_title']
3. jobs_data_cleaned['processed_description']


In [ ]:
def add_token_count_column(df, column_name):
    """
    Adds a new column to the DataFrame with the token count for each entry in the specified column.
    This function creates a copy of the DataFrame to avoid 'SettingWithCopyWarning'.

    Args:
    df (pd.DataFrame): The DataFrame to process.
    column_name (str): The name of the column for which to count tokens.

    Returns:
    pd.DataFrame: DataFrame with an additional column 'token_count'.
    """
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in DataFrame")

    # Creating a copy of the DataFrame to avoid modifying a slice
    df_copy = df.copy()

    # Tokenize each entry in the specified column and count the number of tokens
    df_copy['token_count'] = df_copy[column_name].apply(lambda x: len(word_tokenize(x)) if pd.notnull(x) else 0)

    return df_copy


In [ ]:
jobs_data_cleaned_with_tokens = add_token_count_column(jobs_data_cleaned, column_name='processed_description')


In [ ]:
jobs_data_cleaned_with_tokens

,Unnamed: 0,index,title,company_name,location,via,description,extensions,job_id,thumbnail,...,salary_avg,salary_min,salary_max,salary_hourly,salary_yearly,salary_standardized,description_tokens,processed_description,processed_title,token_count
0,0,0,Data Analyst,Chloeta,"Oklahoma City, OK",via ZipRecruiter,Job Summary: The Data Analyst oversees data pr...,"['21 hours ago', 'Full-time', 'Health insuranc...",eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QiLCJodGlkb2...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"['r', 'python']",job summary data analyst oversees data product...,data analyst,600
1,1,1,Junior Data Analyst/Scientist Role - Contract ...,Upwork,Anywhere,via Upwork,"Company\n\nThe TAC Index provides independent,...","['17 hours ago', 'Work from home', 'Contractor...",eyJqb2JfdGl0bGUiOiJKdW5pb3IgRGF0YSBBbmFseXN0L1...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"['python', 'r', 'mysql', 'matplotlib', 'matlab...",company tac index provides independent accurat...,junior data analystscientist role contract hire,152
2,2,2,Data Analyst,ATC,United States,via LinkedIn,Job Title: Entry Level Business Analyst / Prod...,"['12 hours ago', 'Full-time', 'Health insurance']",eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QiLCJodGlkb2...,https://encrypted-tbn0.gstatic.com/images?q=tb...,...,NaN,NaN,NaN,NaN,NaN,NaN,[],job title entry level business analyst product...,data analyst,128
3,3,3,Data Analyst,Guidehouse,"Topeka, KS",via Nexxt,Job Family :\n\nData Science & Analysis (Digit...,"['10 hours ago', 'Full-time', 'Health insuranc...",eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QiLCJodGlkb2...,https://encrypted-tbn0.gstatic.com/images?q=tb...,...,NaN,NaN,NaN,NaN,NaN,NaN,"['power_bi', 'powerpoint', 'excel', 'word', 'r...",job family data science analysis digital trave...,data analyst,514
4,4,4,Data Analyst,AnMed Health LLC,Anywhere,via LinkedIn,"AnMed is a dynamic, comprehensive health syste...","['18 hours ago', 'Work from home', 'Part-time'...",eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QiLCJodGlkb2...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,[],anmed dynamic comprehensive health system ande...,data analyst,153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59637,59637,1316,"Data Analyst, Supply Chain",The Home Depot,United States,via NRF Job Board - National Retail Federation,Position Purpose:\n\nA Data Analyst Supply Cha...,"['14 hours ago', 'Full-time']",eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QsIFN1cHBseS...,https://encrypted-tbn0.gstatic.com/images?q=tb...,...,NaN,NaN,NaN,NaN,NaN,NaN,['sql'],position purpose data analyst supply chain lev...,data analyst supply chain,323
59638,59638,1317,Planning Analyst,Microsoft,United States,via Jobs | The FutureList Job Board - The Futu...,The Cloud Operations + Innovation (CO+I) organ...,"['9 hours ago', 'Full-time', 'Health insurance']",eyJqb2JfdGl0bGUiOiJQbGFubmluZyBBbmFseXN0IiwiaH...,https://encrypted-tbn0.gstatic.com/images?q=tb...,...,NaN,NaN,NaN,NaN,NaN,NaN,"['excel', 'sql']",cloud operation innovation coi organization lo...,planning analyst,497
59639,59639,1318,"Data Analyst, Business Services",Community College of Baltimore County,United States,via Higher Education Recruitment Consortium (H...,The purpose of this class is to perform profes...,"['20 hours ago', 'Full-time']",eyJ1dWxlIjoidytDQUlRSUNJTlZXNXBkR1ZrSUZOMFlYUm...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"['sql', 'power_bi', 'excel']",purpose class perform professional level techn...,data analyst business service,130
59640,59640,1319,Data Analyst 2,ORAU,Anywhere,via Oak Ridge Associated Universities | Career...,Overview\n\nAppointment Type: Full-Time Regula...,"['20 hours ago', 'Work from home', 'Full-time'...",eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QgMiIsImh0aW...,https://encrypted-tbn0.gstatic.com/images?q=tb...,...,NaN,NaN,NaN,NaN,NaN,NaN,"['spreadsheet', 'excel', 'power_bi', 'r', 'sql...",overview appointment type fulltime regular ftr...,data analyst,612


In [ ]:
resume_data_cleaned_with_tokens = add_token_count_column(resume_data_cleaned, column_name='processed_resume')


In [ ]:
resume_data_cleaned_with_tokens

,Category,Resume,processed_resume,token_count
0,Data Science,Skills * Programming Languages: Python (pandas...,skill programming language python panda numpy ...,488
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...,education detail may may uitrgpv data scientis...,118
2,Data Science,"Areas of Interest Deep Learning, Control Syste...",area interest deep learning control system des...,180
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...,skill r python sap hana tableau sap hana sql s...,715
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab...",education detail mca ymcaust faridabad haryana...,49
...,...,...,...,...
894,Testing,Computer Skills: â¢ Proficient in MS office (...,computer skill proficient m office word basic ...,218
895,Testing,â Willingness to accept the challenges. â ...,willingness accept challenge positive thinking...,209
896,Testing,"PERSONAL SKILLS â¢ Quick learner, â¢ Eagerne...",personal skill quick learner eagerness learn n...,316
897,Testing,COMPUTER SKILLS & SOFTWARE KNOWLEDGE MS-Power ...,computer skill software knowledge mspower poin...,87


**Dropping all unnecessary features**

In [ ]:
# Dropping unnecessary columns from jobs data
jobs_data_final = jobs_data_cleaned_with_tokens[['processed_title', 'processed_description', 'token_count']]

# Dropping unnecessary columns from resume data
resume_data_final = resume_data_cleaned_with_tokens[['processed_resume', 'token_count']]


In [ ]:
jobs_data_final.shape

(41562, 3)

In [ ]:
resume_data_final.shape

(166, 2)

In [ ]:
jobs_data_final

,processed_title,processed_description,token_count
0,data analyst,job summary data analyst oversees data product...,600
1,junior data analystscientist role contract hire,company tac index provides independent accurat...,152
2,data analyst,job title entry level business analyst product...,128
3,data analyst,job family data science analysis digital trave...,514
4,data analyst,anmed dynamic comprehensive health system ande...,153
...,...,...,...
59637,data analyst supply chain,position purpose data analyst supply chain lev...,323
59638,planning analyst,cloud operation innovation coi organization lo...,497
59639,data analyst business service,purpose class perform professional level techn...,130
59640,data analyst,overview appointment type fulltime regular ftr...,612


In [ ]:
resume_data_final

,processed_resume,token_count
0,skill programming language python panda numpy ...,488
1,education detail may may uitrgpv data scientis...,118
2,area interest deep learning control system des...,180
3,skill r python sap hana tableau sap hana sql s...,715
4,education detail mca ymcaust faridabad haryana...,49
...,...,...
894,computer skill proficient m office word basic ...,218
895,willingness accept challenge positive thinking...,209
896,personal skill quick learner eagerness learn n...,316
897,computer skill software knowledge mspower poin...,87


**Fearues**


In [ ]:
jobs_data_final.info()


<class 'pandas.core.frame.DataFrame'>
Index: 41513 entries, 0 to 59641
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   processed_title        41513 non-null  object
 1   processed_description  41513 non-null  object
 2   token_count            41513 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.3+ MB


In [ ]:
resume_data_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 166 entries, 0 to 898
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   processed_resume  166 non-null    object
 1   token_count       166 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 3.9+ KB


# Let's load Custom Summarization model

In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer
import torch

class TextSummarizer:
    """
    A text summarization class that uses a fine-tuned BART model to summarize text.

    Attributes:
        device (str): Device to run the model on, either 'cuda' or 'cpu'.
        model (BartForConditionalGeneration): The loaded BART model.
        tokenizer (BartTokenizer): The tokenizer for the BART model.
    """

    def __init__(self, model_name):
        """
        Initializes the TextSummarizer with a specified BART model.

        Args:
            model_name (str): The name or path of the fine-tuned BART model.
        """
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = BartForConditionalGeneration.from_pretrained(model_name).to(self.device)
        self.tokenizer = BartTokenizer.from_pretrained(model_name)

    def summarize(self, text, max_input_length=1024, max_output_length=150, min_output_length=40):
        """
        Summarizes the given text using the fine-tuned BART model.

        Args:
            text (str): The text to be summarized.
            max_input_length (int): The maximum length of the input text in tokens.
            max_output_length (int): The maximum length of the summary text in tokens.
            min_output_length (int): The minimum length of the summary text in tokens.

        Returns:
            str: The summarized text.
        """
        inputs = self.tokenizer([text], max_length=max_input_length, return_tensors='pt', truncation=True)
        summary_ids = self.model.generate(
            inputs['input_ids'].to(self.device),
            max_length=max_output_length,
            min_length=min_output_length,
            length_penalty=2.0,
            num_beams=4,
            early_stopping=True
        )
        return self.tokenizer.decode(summary_ids[0], skip_special_tokens=True)


In [ ]:
summarizer = TextSummarizer("geekradius/bart-large-cnn-fintetuned-samsum-repo")


config.json:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/328 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/999k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

In [ ]:
!pip install tqdm


In [ ]:
from tqdm import tqdm
import pandas as pd

def batch_summarize(df, text_col, summarizer, batch_size=10, output_col=None):
    """
    Summarizes text in batches.

    Args:
        df (pd.DataFrame): The DataFrame containing text to summarize.
        text_col (str): The column in the DataFrame with text to summarize.
        summarizer: The summarizer object or function.
        batch_size (int): The size of each batch for summarization.
        output_col (str, optional): The name of the output column for summarized text.
                                   If None, defaults to text_col.

    Returns:
        pd.DataFrame: DataFrame with summarized text in the specified output column.
    """
    summarized_texts = []

    # Use the text_col as output_col if not specified
    if output_col is None:
        output_col = text_col

    # Iterate through the DataFrame in batches
    for start_idx in tqdm(range(0, len(df), batch_size), desc="Summarizing"):
        end_idx = start_idx + batch_size
        batch = df[text_col][start_idx:end_idx]

        # Summarize each batch
        summarized_batch = [summarizer.summarize(text) for text in batch]
        summarized_texts.extend(summarized_batch)

    # Create a new DataFrame with the summarized text
    return pd.DataFrame({output_col: summarized_texts})


# Summarize!

In [ ]:
# Summarize the top 100 'processed_description' of jobs_data_final
top_jobs_data = jobs_data_final.head(100)

# Summarize jobs description
jobs_data_summarized = batch_summarize(top_jobs_data, 'processed_description', summarizer, batch_size=10, output_col='summarized_description')

# Summarize all 'processed_resume' in resume_data_final
resume_data_summarized = batch_summarize(resume_data_final, 'processed_resume', summarizer, batch_size=10, output_col='summarized_resume')


Summarizing: 100%|██████████| 17/17 [55:47<00:00, 196.94s/it]


In [ ]:
# # Save jobs_data_summarized to a CSV file
# jobs_data_summarized.to_csv('jobs_data_summarized.csv', index=False)

# # Save resume_data_summarized to a Pickle file
# resume_data_summarized.to_pickle('resume_data_summarized.pkl')
import pandas as pd
import pickle

# Load summarized job data from CSV file
jobs_data_summarized = pd.read_csv('jobs_data_summarized.csv')

# Load summarized resume data from Pickle file
resume_data_summarized = pd.read_pickle('resume_data_summarized.pkl')

In [ ]:
resume_data_summarized.head(5)
jobs_data_summarized.head(5)

,summarized_description
0,Job summary: Data analyst oversees data produc...
1,"Tac index provides accurate, actionable global..."
2,Job title is entry level business analyst prod...
3,The job is to work as a data analyst for a mod...
4,Anmed is looking for an experienced data analy...


In [ ]:
resume_data_summarized.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166 entries, 0 to 165
Data columns (total 1 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   summarized_resume  166 non-null    object
dtypes: object(1)
memory usage: 1.4+ KB


In [ ]:
jobs_data_summarized.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 1 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   summarized_description  100 non-null    object
dtypes: object(1)
memory usage: 932.0+ bytes


# Encoding with SentenceTransformers

In [ ]:
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
from sentence_transformers import SentenceTransformer
import pandas as pd

class SentenceTransformerEncoder:
    """
    A class to handle sentence encoding using Sentence Transformers, directly working with pandas DataFrames.
    This class encodes text data in a specified DataFrame column into vector representations.

    Attributes:
        model (SentenceTransformer): The Sentence Transformer model used for encoding.
    """

    def __init__(self, model_name='all-MiniLM-L6-v2'):
        """
        Initializes the SentenceTransformerEncoder with a specified Sentence Transformer model.

        Args:
            model_name (str): The name of the Sentence Transformer model.
        """
        self.model = SentenceTransformer(model_name)

    def encode_column(self, df, column, batch_size=32, encoded_column_suffix='_encoded'):
        """
        Encodes a specific column in a DataFrame and adds a new column with encoded vectors.

        Args:
            df (pd.DataFrame): The DataFrame containing the texts to encode.
            column (str): The name of the column to encode.
            batch_size (int): The size of each batch for processing.
            encoded_column_suffix (str): Suffix for the new column containing encoded vectors.

        Returns:
            pd.DataFrame: The original DataFrame with an additional column containing encoded vectors.

        Raises:
            ValueError: If the specified column is not found in the DataFrame.
        """
        if column not in df.columns:
            raise ValueError(f"Column '{column}' not found in DataFrame")

        # Encoding the text data in batches
        encoded_vectors = []
        for start_idx in range(0, len(df), batch_size):
            end_idx = min(start_idx + batch_size, len(df))
            batch_texts = df[column][start_idx:end_idx].tolist()
            batch_encoded = self.model.encode(batch_texts, show_progress_bar=True)
            encoded_vectors.extend(batch_encoded)

        # Adding the encoded vectors as a new column in the DataFrame
        df[column + encoded_column_suffix] = encoded_vectors
        return df

# # Example Usage
# encoder = SentenceTransformerEncoder()

# # Encoding the summarized job descriptions
# jobs_data_summarized_and_encoded = encoder.encode_column(jobs_data_summarized, 'summarized_description')

# # Encoding the summarized resumes
# resume_data_summarized_and_encoded = encoder.encode_column(resume_data_summarized, 'summarized_resume')


In [ ]:
# Example Usage
encoder = SentenceTransformerEncoder(model_name='all-MiniLM-L6-v2')

# Encoding the summarized job descriptions
jobs_data_summarized_and_encoded = encoder.encode_column(jobs_data_summarized, 'summarized_description')

# Encoding the summarized resumes
resume_data_summarized_and_encoded = encoder.encode_column(resume_data_summarized, 'summarized_resume')


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
jobs_data_summarized_and_encoded

,summarized_description,summarized_description_encoded
0,Job summary: Data analyst oversees data produc...,"[-0.014705562, 0.025372537, -0.044520564, 0.03..."
1,"Tac index provides accurate, actionable global...","[-0.041204665, -0.057401866, -0.013391327, 0.0..."
2,Job title is entry level business analyst prod...,"[-0.06329379, 0.015474094, -0.03660898, 0.0127..."
3,The job is to work as a data analyst for a mod...,"[-0.035364475, 0.033276975, -0.060001172, 0.07..."
4,Anmed is looking for an experienced data analy...,"[-0.038958125, -0.010983811, -0.039488085, 0.0..."
...,...,...
95,World gate recruiter team is looking for a ful...,"[-0.040161487, -0.039902464, -0.024759982, 0.0..."
96,Teksystems needs a new data analyst. The ideal...,"[-0.02775288, 0.027919639, -0.07045641, 0.0169..."
97,Walmart's global tech team is looking for a ne...,"[0.0043830825, -0.01946177, 0.039011717, 0.006..."
98,Data analyst is needed to work for a business ...,"[-0.027573023, -0.008233752, -0.010068408, 0.0..."


In [ ]:
resume_data_summarized_and_encoded

,summarized_resume,summarized_resume_encoded
0,skill programming language python panda numpy ...,"[-0.06653203, 0.018028302, -0.045124486, -0.06..."
1,"Education detail may may uitrGPv, data scienti...","[-0.11100649, 0.013230324, -0.03152348, 0.0029..."
2,area interest deep learning control system des...,"[-0.09938316, -0.0748482, 0.014191928, 0.01172..."
3,"skill r python sap hana tableau, sana, hana, p...","[-0.026053384, -0.06500236, -0.0040796804, 0.0..."
4,"Data structure exprience less year month c, py...","[-0.049389794, -0.011031546, 0.004312771, 0.00..."
...,...,...
161,Computer skill proficient m office word basic ...,"[-0.039111517, 0.05445337, 0.012831537, 0.0343..."
162,Dongare mandakini murlidhar is the name of a c...,"[-0.0864619, 0.0048061446, -0.056528162, -0.05..."
163,Dhanraj wagheducation detail is a company that...,"[-0.05996504, 0.12965296, -0.041374337, 0.0416..."
164,Computer skill software knowledge mspower poin...,"[-0.058406144, 0.0881397, -0.01209699, -0.0576..."


In [ ]:
jobs_data_final

,processed_title,processed_description,token_count
0,data analyst,job summary data analyst oversees data product...,600
1,junior data analystscientist role contract hire,company tac index provides independent accurat...,152
2,data analyst,job title entry level business analyst product...,128
3,data analyst,job family data science analysis digital trave...,514
4,data analyst,anmed dynamic comprehensive health system ande...,153
...,...,...,...
59637,data analyst supply chain,position purpose data analyst supply chain lev...,323
59638,planning analyst,cloud operation innovation coi organization lo...,497
59639,data analyst business service,purpose class perform professional level techn...,130
59640,data analyst,overview appointment type fulltime regular ftr...,612


In [ ]:
resume_data_final

,processed_resume,token_count
0,skill programming language python panda numpy ...,488
1,education detail may may uitrgpv data scientis...,118
2,area interest deep learning control system des...,180
3,skill r python sap hana tableau sap hana sql s...,715
4,education detail mca ymcaust faridabad haryana...,49
...,...,...
894,computer skill proficient m office word basic ...,218
895,willingness accept challenge positive thinking...,209
896,personal skill quick learner eagerness learn n...,316
897,computer skill software knowledge mspower poin...,87


In [ ]:
# Combine the jobs data
jobs_combined = pd.merge(
    jobs_data_final,
    jobs_data_summarized_and_encoded[['summarized_description', 'summarized_description_encoded']],
    left_index=True, right_index=True)

# Combine the resume data
resume_combined = pd.merge(
    resume_data_final,
    resume_data_summarized_and_encoded[['summarized_resume', 'summarized_resume_encoded']],
    left_index=True, right_index=True)

# Reset index of DataFrame
jobs_combined.reset_index(drop=True, inplace=True)
resume_combined.reset_index(drop=True, inplace=True)

In [ ]:
jobs_combined.head(5)

,processed_title,processed_description,token_count,summarized_description,summarized_description_encoded
0,data analyst,job summary data analyst oversees data product...,600,Job summary: Data analyst oversees data produc...,"[-0.014705562, 0.025372537, -0.044520564, 0.03..."
1,junior data analystscientist role contract hire,company tac index provides independent accurat...,152,"Tac index provides accurate, actionable global...","[-0.041204665, -0.057401866, -0.013391327, 0.0..."
2,data analyst,job title entry level business analyst product...,128,Job title is entry level business analyst prod...,"[-0.06329379, 0.015474094, -0.03660898, 0.0127..."
3,data analyst,job family data science analysis digital trave...,514,The job is to work as a data analyst for a mod...,"[-0.035364475, 0.033276975, -0.060001172, 0.07..."
4,data analyst,anmed dynamic comprehensive health system ande...,153,Anmed is looking for an experienced data analy...,"[-0.038958125, -0.010983811, -0.039488085, 0.0..."


In [ ]:
resume_combined.head(5)

,processed_resume,token_count,summarized_resume,summarized_resume_encoded
0,skill programming language python panda numpy ...,488,skill programming language python panda numpy ...,"[-0.06653203, 0.018028302, -0.045124486, -0.06..."
1,education detail may may uitrgpv data scientis...,118,"Education detail may may uitrGPv, data scienti...","[-0.11100649, 0.013230324, -0.03152348, 0.0029..."
2,area interest deep learning control system des...,180,area interest deep learning control system des...,"[-0.09938316, -0.0748482, 0.014191928, 0.01172..."
3,skill r python sap hana tableau sap hana sql s...,715,"skill r python sap hana tableau, sana, hana, p...","[-0.026053384, -0.06500236, -0.0040796804, 0.0..."
4,education detail mca ymcaust faridabad haryana...,49,"Data structure exprience less year month c, py...","[-0.049389794, -0.011031546, 0.004312771, 0.00..."


# Upload to Vector Databae - Qdrant

In [ ]:
!pip install qdrant-client


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.7/306.7 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 18.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.6
    Uninstalling protobuf-4.25.6:
      Successfully uninstalled protobuf-4.25.6


In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams, Distance, Record, Filter
from random import uniform


class QdrantInterface:
    """
    A class for interfacing with the Qdrant vector database.

    Attributes:
        client (QdrantClient): Client instance for interacting with Qdrant.
        vector_dimension (int): Dimension of the vectors used in the collection.
    """

    """
    A class for interfacing with the Qdrant vector database.
    ...
    """
    def __init__(self, url, api_key, vector_dimension):
        """
        Initializes the QdrantInterface with the specified Qdrant URL, API key, and vector dimension.

        Args:
            url (str): Full URL of the Qdrant server.
            api_key (str): API key for Qdrant.
            vector_dimension (int): Dimension of vectors to be stored in Qdrant.
        """
        self.client = QdrantClient(url=url, api_key=api_key)
        self.vector_dimension = vector_dimension

    def create_collection(self, collection_name, distance_metric=Distance.COSINE):
        """
        Creates or recreates a collection in Qdrant.

        Args:
            collection_name (str): Name of the collection.
            distance_metric (Distance): Distance metric for vector comparisons.
        """
        self.client.recreate_collection(
            collection_name=collection_name,
            vectors_config=VectorParams(size=self.vector_dimension, distance=distance_metric)
        )

    def save_to_qdrant(self, df, collection_name, vector_col, payload_cols, batch_size=100):
        """
        Saves a DataFrame to Qdrant in batches.

        Args:
            df (pd.DataFrame): DataFrame containing data to save.
            collection_name (str): Name of the collection in Qdrant.
            vector_col (str): Name of the column containing vectors.
            payload_cols (list[str]): List of column names to include as payload.
            batch_size (int): Number of records to process in each batch.
        """

        for start_idx in range(0, len(df), batch_size):
            end_idx = min(start_idx + batch_size, len(df))
            batch = df.iloc[start_idx:end_idx]
            records = []
            for idx, row in batch.iterrows():
                # Debug print
                print(f"Index: {idx}, Vector Type: {type(row[vector_col])}, First 10 Elements: {row[vector_col][:10]}")
                record = Record(
                    id=idx,
                    vector=row[vector_col],
                    payload={col: row[col] for col in payload_cols}
                )
                records.append(record)
            self.client.upload_records(collection_name=collection_name, records=records)



    def retrieve_specific_records(self, collection_name, ids):
        """
        Retrieves specific records by their IDs from a Qdrant collection.

        Args:
            collection_name (str): The name of the collection.
            ids (list): List of record IDs to retrieve.

        Returns:
            List of specific records from the collection.
        """
        return self.client.retrieve(collection_name=collection_name, ids=ids)

    def view_sample_records(self, collection_name, vector_dimension, limit=10):
        """
        Retrieves a sample of records from a Qdrant collection using a dummy search.

        Args:
            collection_name (str): The name of the collection.
            vector_dimension (int): Dimension of vectors in the collection.
            limit (int): The number of records to retrieve.

        Returns:
            List of sample records from the collection.
        """
        # Generate a random vector
        random_vector = [uniform(-1, 1) for _ in range(vector_dimension)]

        # Perform a dummy search
        return self.client.search(
            collection_name=collection_name,
            query_vector=random_vector,
            limit=limit
        )

    def match_resumes_to_jobs(self, resume_vector, top_k=10):
        """
        Matches a given resume vector to job postings.

        Args:
            resume_vector (list): The vector representation of a resume.
            top_k (int): Number of top similar matches to return.

        Returns:
            List of matched job postings with similarity scores.
        """
        hits = self.client.search(
            collection_name="jobs",
            query_vector=resume_vector,
            limit=top_k,
            with_payload=True
        )
        return [(hit.payload, hit.score) for hit in hits]

    def match_jobs_to_resumes(self, job_vector, top_k=10):
        """
        Matches a given job vector to resumes.

        Args:
            job_vector (list): The vector representation of a job posting.
            top_k (int): Number of top similar matches to return.

        Returns:
            List of tuples containing matched resumes and their similarity scores.
        """
        hits = self.client.search(
            collection_name="resumes",
            query_vector=job_vector,
            limit=top_k,
            with_payload=True
        )
        return [(hit.payload, hit.score) for hit in hits]



In [ ]:
vector_dimension = encoder.model.get_sentence_embedding_dimension()


In [ ]:
vector_dimension

384

In [ ]:


vector_dimension = encoder.model.get_sentence_embedding_dimension()
qdrant_interface = QdrantInterface(QUADRANT_ENDPOINT, QUADRANT_API_KEY, vector_dimension)
qdrant_interface.create_collection('jobs', Distance.COSINE)
qdrant_interface.create_collection('resumes', Distance.COSINE)
# Add unique IDs to your DataFrames before uploading
import uuid

# Add unique IDs to jobs_combined
jobs_combined['job_id'] = [str(uuid.uuid4()) for _ in range(len(jobs_combined))]

# Add unique IDs to resume_combined
resume_combined['resume_id'] = [str(uuid.uuid4()) for _ in range(len(resume_combined))]
# Function to ensure vectors are in list format
def ensure_list_format(df, vector_col):
    df[vector_col] = df[vector_col].apply(lambda x: x.tolist() if hasattr(x, 'tolist') else x)
    return df

# Ensure vectors are in the correct format before uploading
jobs_combined = ensure_list_format(jobs_combined, 'summarized_description_encoded')
resume_combined = ensure_list_format(resume_combined, 'summarized_resume_encoded')



<ipython-input-41-3d954d70375d>:39: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  self.client.recreate_collection(


In [ ]:
jobs_combined.head(5)

,processed_title,processed_description,token_count,summarized_description,summarized_description_encoded,job_id
0,data analyst remote year,employer seeking data analyst join team make p...,229,Job summary: Data analyst oversees data produc...,"[-0.01470556203275919, 0.025372536852955818, -...",4a33b9d7-5cdb-46ba-b519-d1c0b2f15969
1,data analyst,salary semimonthly salary based education expe...,276,"Tac index provides accurate, actionable global...","[-0.04120466485619545, -0.0574018657207489, -0...",eead3310-0938-42bf-96bc-20feffc7eaf3
2,sql data analyst remote,healthcare sql data analyst professional work ...,311,Job title is entry level business analyst prod...,"[-0.06329379230737686, 0.015474094077944756, -...",8d4ebb7c-e681-46c0-8b1a-63eb7ca8e710
3,data analyst,job title entry level business analyst product...,128,The job is to work as a data analyst for a mod...,"[-0.03536447510123253, 0.03327697515487671, -0...",05534a9a-6cad-44cd-931c-9a9cd427801c
4,senior data analyst remote,position purpose senior data analyst work team...,302,Anmed is looking for an experienced data analy...,"[-0.038958124816417694, -0.010983810760080814,...",ee29e48f-e62d-485e-99ce-a4c8eeff9f01


In [ ]:
sample_job_vector = jobs_combined['summarized_description_encoded'].iloc[0]
print(type(sample_job_vector))
print(sample_job_vector[:10])  # Print first 10 elements of the vector


<class 'list'>
[-0.01470556203275919, 0.025372536852955818, -0.04452056437730789, 0.03583376854658127, -0.0074485535733401775, -0.042682237923145294, 0.013360329903662205, -0.06411147117614746, -0.05445580184459686, -0.003336200723424554]


In [ ]:
# Now upload to Qdrant
qdrant_interface.save_to_qdrant(jobs_combined, 'jobs', 'summarized_description_encoded', ['processed_title', 'processed_description', 'token_count', 'summarized_description','job_id'])
qdrant_interface.save_to_qdrant(resume_combined, 'resumes', 'summarized_resume_encoded', ['processed_resume', 'token_count', 'summarized_resume','resume_id'])


Index: 0, Vector Type: <class 'list'>, First 10 Elements: [-0.01470556203275919, 0.025372536852955818, -0.04452056437730789, 0.03583376854658127, -0.0074485535733401775, -0.042682237923145294, 0.013360329903662205, -0.06411147117614746, -0.05445580184459686, -0.003336200723424554]
Index: 1, Vector Type: <class 'list'>, First 10 Elements: [-0.04120466485619545, -0.0574018657207489, -0.013391327112913132, 0.08617253601551056, 0.022698070853948593, -0.0607285350561142, 0.048123497515916824, -0.038525793701410294, -0.0011187632335349917, 0.006021213252097368]
Index: 2, Vector Type: <class 'list'>, First 10 Elements: [-0.06329379230737686, 0.015474094077944756, -0.0366089791059494, 0.01279176864773035, -0.08607363700866699, -0.0078873410820961, 0.029287438839673996, -0.024566391482949257, -0.0478435680270195, -0.12940917909145355]
Index: 3, Vector Type: <class 'list'>, First 10 Elements: [-0.03536447510123253, 0.03327697515487671, -0.06000117212533951, 0.07073717564344406, -0.08640295267105

<ipython-input-41-3d954d70375d>:69: DeprecationWarning: `upload_records` is deprecated, use `upload_points` instead
  self.client.upload_records(collection_name=collection_name, records=records)


Index: 0, Vector Type: <class 'list'>, First 10 Elements: [-0.06653203070163727, 0.018028302118182182, -0.04512448608875275, -0.06566372513771057, 0.05396505445241928, -0.10004422068595886, 0.04261768236756325, 0.01096956804394722, -0.0020046215504407883, 0.04652181640267372]
Index: 1, Vector Type: <class 'list'>, First 10 Elements: [-0.1110064908862114, 0.013230323791503906, -0.031523481011390686, 0.0029658954590559006, 0.011477207764983177, -0.06325867772102356, -0.08687053620815277, 0.007702519651502371, -0.1462545245885849, 0.03142821416258812]
Index: 2, Vector Type: <class 'list'>, First 10 Elements: [-0.0993831604719162, -0.07484819740056992, 0.014191928319633007, 0.011727147735655308, -0.044331349432468414, -0.10225855559110641, -0.024494566023349762, 0.08790478110313416, -0.11956935375928879, -0.02876036986708641]
Index: 3, Vector Type: <class 'list'>, First 10 Elements: [-0.026053383946418762, -0.06500235944986343, -0.004079680424183607, 0.02044352889060974, -0.061169452965259

In [ ]:
# View sample records from the 'jobs' collection
sample_jobs_records = qdrant_interface.view_sample_records('jobs', vector_dimension, limit=2)

<ipython-input-41-3d954d70375d>:102: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  return self.client.search(


In [ ]:
sample_jobs_records

[ScoredPoint(id=72, version=1, score=0.09436157, payload={'processed_title': 'data qualitydata control specialist', 'processed_description': 'job title data qualitydata control specialist job location remote job duration longterm job description tracking monitoring data quality indicator across bank data quality report dashboard compliance data quality framework performing root cause analysis prioritized data quality issue determine appropriate fix forward fix backward action remediation plan working key stakeholder align expected turnaround time remediation gaining approval suggested remediation action coordinating appropriate action team responsible executing action monitor remediation action till closure participate central data cleansing activity accelerate data quality remediation participate data quality improvement initiative project musthave year experience data quality management practice data profiling dq assessment data cleansing etc data quality related solution skill knowl

In [ ]:
# Retrieve specific records by IDs from the 'jobs' collection
specific_jobs_records = qdrant_interface.retrieve_specific_records('jobs', ids=[1])


In [ ]:
specific_jobs_records

[Record(id=1, payload={'processed_title': 'data analyst', 'processed_description': 'salary semimonthly salary based education experience job location position missouri department natural resource division state park business service program located e elm jefferson city mo hour pm monday friday youll love position looking people join team love serving others working missouri finest example natural cultural resource enjoy youll create tool help make data driven decision youll become familiar data element division work illustrate element variety reporting application youll create review analyze financial report reflecting expenditure revenue project tracking appropriation budget allocation provide information management assist making data driven decision develop analysis fiscal information division reflecting projected spending forecasted revenue well identify trend analysis identify area concern area success suggest operational change track impact implemented develop reporting dashboard 

# Matching:

With the encoded representations of job descriptions and resumes, we can then proceed to match them based on their similarity.

This step typically involves calculating similarity scores between resume encodings and job description encodings, which can be achieved through methods like cosine similarity.


In [ ]:
# Example job vector from the jobs_combined DataFrame
example_job_vector = jobs_combined['summarized_description_encoded'].iloc[43]

In [ ]:
jobs_combined['summarized_description'][43]

'Centene is an equal opportunity employer committed to diversity value. It provides a comprehensive benefit package including holiday health insurance coverage, stock purchase plan, and tuition reimbursement. It also provides a modern business intelligence visualization tool. It is based on a data mining and analysis.'

In [ ]:
# Find top 5 matching resumes for the example job
matched_resumes = qdrant_interface.match_jobs_to_resumes(example_job_vector, top_k=5)
for resume, score in matched_resumes:
    print(f"Matched Resume: {resume['summarized_resume']}, Score: {score}")


<ipython-input-41-3d954d70375d>:138: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = self.client.search(


Matched Resume: technical skill set big data ecosystem hadoop hdfs hbase map reduce sqoop hive pig sparkcore flume language scala corejava sql plsql sell scripting etl tool informatica power centerx talend tool eclipse platform window family linux unix cloudera database. , Score: 0.3264514
Matched Resume: technical skill programming language c c c java net jee, hfsh, hdinsight, scala sqoop, kmeans clustering random forest, nave bayes etc. , Score: 0.28867483
Matched Resume: Education detail btech electronics instrumentation engineering is working on a project involving a payment gateway, an employee time tracking task assignment tracker, a real estate agent website and a chat group chat app., Score: 0.27163857
Matched Resume: skill python tableau data visualization r studio machine learning statistic iabac certified data scientist with experience year managing business data science consulting leading innovation project bringing business idea working real world solution., Score: 0.27101

**start from here**

In [ ]:
!pip install pyngrok
!pip install qdrant-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.7/306.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 19.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.6
    Uninstalling protobuf-4.25.6:
      Successfully uninstalled protobuf-4.25.6


Running The Flask Server

In [ ]:
# Setting Up Preprocessor
# Libraries

import nltk
import zipfile
import os
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

def setup_nltk_resources():
    """
    Sets up the custom NLTK data path and downloads necessary resources.
    Downloads 'wordnet' for lemmatization, 'stopwords' for stopwords removal,
    and 'punkt' for sentence tokenization.
    """
    nltk_data_path = "/root/nltk_data"
    nltk.data.path.append(nltk_data_path)

    nltk.download('wordnet', download_dir=nltk_data_path)
    nltk.download('stopwords', download_dir=nltk_data_path)
    nltk.download('punkt', download_dir=nltk_data_path)
    nltk.download('punkt_tab', download_dir=nltk_data_path) # Download the missing resource

def unzip_nltk_resource(zip_path, extract_to):
    """
    Unzips an NLTK resource file to a specified directory.

    Args:
    zip_path (str): The path to the zipped NLTK resource file.
    extract_to (str): The directory where the contents of the zip file will be extracted.
    """
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

def preprocess_text(text):
    """
    Preprocesses a given text string for NLP tasks. This includes cleaning the text,
    tokenizing, removing stopwords, and lemmatizing the words.

    Args:
    text (str): The text string to preprocess.

    Returns:
    str: The preprocessed text.
    """
    if not text:
        return ""
    text = re.sub(r'[\r\n\t]+', ' ', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()

    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]

    lemmatizer = WordNetLemmatizer()
    lemmatized_text = [lemmatizer.lemmatize(word) for word in filtered_tokens]

    return ' '.join(lemmatized_text)
# Setup NLTK resources
setup_nltk_resources()

# Unzip wordnet
corpora_path = "/root/nltk_data/corpora"
wordnet_zip = os.path.join(corpora_path, "wordnet.zip")
unzip_nltk_resource(wordnet_zip, corpora_path)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
# Setting Up Summarizer
from transformers import BartForConditionalGeneration, BartTokenizer
import torch

class TextSummarizer:
    """
    A text summarization class that uses a fine-tuned BART model to summarize text.

    Attributes:
        device (str): Device to run the model on, either 'cuda' or 'cpu'.
        model (BartForConditionalGeneration): The loaded BART model.
        tokenizer (BartTokenizer): The tokenizer for the BART model.
    """

    def __init__(self, model_name):
        """
        Initializes the TextSummarizer with a specified BART model.

        Args:
            model_name (str): The name or path of the fine-tuned BART model.
        """
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = BartForConditionalGeneration.from_pretrained(model_name).to(self.device)
        self.tokenizer = BartTokenizer.from_pretrained(model_name)

    def summarize(self, text, max_input_length=1024, max_output_length=150, min_output_length=40):
        """
        Summarizes the given text using the fine-tuned BART model.

        Args:
            text (str): The text to be summarized.
            max_input_length (int): The maximum length of the input text in tokens.
            max_output_length (int): The maximum length of the summary text in tokens.
            min_output_length (int): The minimum length of the summary text in tokens.

        Returns:
            str: The summarized text.
        """
        inputs = self.tokenizer([text], max_length=max_input_length, return_tensors='pt', truncation=True)
        summary_ids = self.model.generate(
            inputs['input_ids'].to(self.device),
            max_length=max_output_length,
            min_length=min_output_length,
            length_penalty=2.0,
            num_beams=4,
            early_stopping=True
        )
        return self.tokenizer.decode(summary_ids[0], skip_special_tokens=True)
summarizer = TextSummarizer("geekradius/bart-large-cnn-fintetuned-samsum-repo")
summarizer

config.json:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/328 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/999k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

In [ ]:
# Setting Up Encoder
from sentence_transformers import SentenceTransformer
import pandas as pd

class SentenceTransformerEncoder:
    """
    A class to handle sentence encoding using Sentence Transformers, directly working with pandas DataFrames.
    This class encodes text data in a specified DataFrame column into vector representations.

    Attributes:
        model (SentenceTransformer): The Sentence Transformer model used for encoding.
    """

    def __init__(self, model_name='all-MiniLM-L6-v2'):
        """
        Initializes the SentenceTransformerEncoder with a specified Sentence Transformer model.

        Args:
            model_name (str): The name of the Sentence Transformer model.
        """
        self.model = SentenceTransformer(model_name)

    def encode_column(self, df, column, batch_size=32, encoded_column_suffix='_encoded'):
        """
        Encodes a specific column in a DataFrame and adds a new column with encoded vectors.

        Args:
            df (pd.DataFrame): The DataFrame containing the texts to encode.
            column (str): The name of the column to encode.
            batch_size (int): The size of each batch for processing.
            encoded_column_suffix (str): Suffix for the new column containing encoded vectors.

        Returns:
            pd.DataFrame: The original DataFrame with an additional column containing encoded vectors.

        Raises:
            ValueError: If the specified column is not found in the DataFrame.
        """
        if column not in df.columns:
            raise ValueError(f"Column '{column}' not found in DataFrame")

        # Encoding the text data in batches
        encoded_vectors = []
        for start_idx in range(0, len(df), batch_size):
            end_idx = min(start_idx + batch_size, len(df))
            batch_texts = df[column][start_idx:end_idx].tolist()
            batch_encoded = self.model.encode(batch_texts, show_progress_bar=True)
            encoded_vectors.extend(batch_encoded)

        # Adding the encoded vectors as a new column in the DataFrame
        df[column + encoded_column_suffix] = encoded_vectors
        return df
    def encode_text(self, text):
        """
        Encodes a single piece of plain text into a vector representation.

        Args:
            text (str): The text to encode.

        Returns:
            list: The encoded vector representation of the input text.
        """
        if not isinstance(text, str):
            raise ValueError("Input text must be a string")
        return self.model.encode(text)


encoder = SentenceTransformerEncoder(model_name='all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Setting Up Qdrant Interface
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams, Distance, Record, Filter
from random import uniform


class QdrantInterface:
    """
    A class for interfacing with the Qdrant vector database.

    Attributes:
        client (QdrantClient): Client instance for interacting with Qdrant.
        vector_dimension (int): Dimension of the vectors used in the collection.
    """

    """
    A class for interfacing with the Qdrant vector database.
    ...
    """
    def __init__(self, url, api_key, vector_dimension):
        """
        Initializes the QdrantInterface with the specified Qdrant URL, API key, and vector dimension.

        Args:
            url (str): Full URL of the Qdrant server.
            api_key (str): API key for Qdrant.
            vector_dimension (int): Dimension of vectors to be stored in Qdrant.
        """
        self.client = QdrantClient(url=url, api_key=api_key)
        self.vector_dimension = vector_dimension

    def create_collection(self, collection_name, distance_metric=Distance.COSINE):
        """
        Creates or recreates a collection in Qdrant.

        Args:
            collection_name (str): Name of the collection.
            distance_metric (Distance): Distance metric for vector comparisons.
        """
        self.client.recreate_collection(
            collection_name=collection_name,
            vectors_config=VectorParams(size=self.vector_dimension, distance=distance_metric)
        )

    def save_to_qdrant(self, df, collection_name, vector_col, payload_cols, batch_size=100):
        """
        Saves a DataFrame to Qdrant in batches.

        Args:
            df (pd.DataFrame): DataFrame containing data to save.
            collection_name (str): Name of the collection in Qdrant.
            vector_col (str): Name of the column containing vectors.
            payload_cols (list[str]): List of column names to include as payload.
            batch_size (int): Number of records to process in each batch.
        """

        for start_idx in range(0, len(df), batch_size):
            end_idx = min(start_idx + batch_size, len(df))
            batch = df.iloc[start_idx:end_idx]
            records = []
            for idx, row in batch.iterrows():
                # Debug print
                print(f"Index: {idx}, Vector Type: {type(row[vector_col])}, First 10 Elements: {row[vector_col][:10]}")
                record = Record(
                    id=idx,
                    vector=row[vector_col],
                    payload={col: row[col] for col in payload_cols}
                )
                records.append(record)
            self.client.upload_records(collection_name=collection_name, records=records)



    def retrieve_specific_records(self, collection_name, ids):
        """
        Retrieves specific records by their IDs from a Qdrant collection.

        Args:
            collection_name (str): The name of the collection.
            ids (list): List of record IDs to retrieve.

        Returns:
            List of specific records from the collection.
        """
        return self.client.retrieve(collection_name=collection_name, ids=ids)

    def view_sample_records(self, collection_name, vector_dimension, limit=10):
        """
        Retrieves a sample of records from a Qdrant collection using a dummy search.

        Args:
            collection_name (str): The name of the collection.
            vector_dimension (int): Dimension of vectors in the collection.
            limit (int): The number of records to retrieve.

        Returns:
            List of sample records from the collection.
        """
        # Generate a random vector
        random_vector = [uniform(-1, 1) for _ in range(vector_dimension)]

        # Perform a dummy search
        return self.client.search(
            collection_name=collection_name,
            query_vector=random_vector,
            limit=limit
        )

    def match_resumes_to_jobs(self, resume_vector, top_k=10):
        """
        Matches a given resume vector to job postings.

        Args:
            resume_vector (list): The vector representation of a resume.
            top_k (int): Number of top similar matches to return.

        Returns:
            List of matched job postings with similarity scores.
        """
        hits = self.client.search(
            collection_name="jobs",
            query_vector=resume_vector,
            limit=top_k,
            with_payload=True
        )
        return [(hit.payload, hit.score) for hit in hits]

    def match_jobs_to_resumes(self, job_vector, top_k=10):
        """
        Matches a given job vector to resumes.

        Args:
            job_vector (list): The vector representation of a job posting.
            top_k (int): Number of top similar matches to return.

        Returns:
            List of tuples containing matched resumes and their similarity scores.
        """
        hits = self.client.search(
            collection_name="resumes",
            query_vector=job_vector,
            limit=top_k,
            with_payload=True
        )
        return [(hit.payload, hit.score) for hit in hits]
vector_dimension = encoder.model.get_sentence_embedding_dimension()

qdrant_interface = QdrantInterface(QUADRANT_ENDPOINT, QUADRANT_API_KEY, vector_dimension)

In [ ]:
from flask import Flask, request, jsonify
import pyngrok.ngrok as ngrok
import numpy as np
import uuid
# Initialize Flask app
app = Flask(__name__)

# API Routes
@app.route('/match', methods=['POST'])
def match_job_resume():
    data = request.json

    if not data or 'job_description' not in data or 'resume_text' not in data:
        return jsonify({'error': 'Missing job_description or resume_text in request'}), 400

    job_description = data['job_description']
    resume_text = data['resume_text']

    # Process job description
    processed_job = preprocess_text(job_description)
    summarized_job = summarizer.summarize(processed_job)
    job_embedding = encoder.encode_text(summarized_job).tolist()

    # Process resume
    processed_resume = preprocess_text(resume_text)
    summarized_resume = summarizer.summarize(processed_resume)
    resume_embedding = encoder.encode_text(summarized_resume).tolist()

    # Calculate direct similarity score
    job_embedding_np = np.array(job_embedding)
    resume_embedding_np = np.array(resume_embedding)

    # Cosine similarity
    similarity = np.dot(job_embedding_np, resume_embedding_np) / (
        np.linalg.norm(job_embedding_np) * np.linalg.norm(resume_embedding_np)
    )

    # Match against existing data (optional)
    matching_jobs = qdrant_interface.match_resumes_to_jobs(resume_embedding, top_k=5)

    return jsonify({
        'direct_match_score': float(similarity),
        'processed_job': processed_job,
        'summarized_job': summarized_job,
        'processed_resume': processed_resume,
        'summarized_resume': summarized_resume,
        'similar_jobs': [{'job': j[0]['summarized_description'], 'score': float(j[1])} for j in matching_jobs]
    })

@app.route('/add_job', methods=['POST'])
def add_job():
    data = request.json

    if not data or 'job_description' not in data or 'job_title' not in data:
        return jsonify({'error': 'Missing job_description or job_title in request'}), 400

    job_title = data['job_title']
    job_description = data['job_description']
    job_id = data.get('job_id', str(abs(hash(job_description)))[:10])  # Generate a unique job ID if not provided

    try:
        # Process job
        processed_title = preprocess_text(job_title)
        processed_job = preprocess_text(job_description)
        summarized_job = summarizer.summarize(processed_job)
        job_embedding = encoder.encode_text(summarized_job).tolist()

        # Store in Qdrant
        from qdrant_client.http.models import PointStruct
        record = PointStruct(
            id=int(job_id),
            vector=job_embedding,
            payload={
                'processed_title': processed_title,
                'processed_description': processed_job,
                'token_count': len(word_tokenize(processed_job)),
                'summarized_description': summarized_job,
                'job_id': job_id  # Store job ID in payload for easy retrieval
            }
        )

        qdrant_interface.client.upsert(
            collection_name="jobs",
            points=[record]
        )

        return jsonify({
            'status': 'success',
            'job_id': job_id,
            'message': 'Job added successfully'
        })
    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/add_resume', methods=['POST'])
def add_resume():
    try:
        data = request.json

        if not data or 'resume_text' not in data:
            return jsonify({'error': 'Missing resume_text in request'}), 400

        resume_text = data['resume_text']
        resume_id = data.get('resume_id', str(abs(hash(resume_text)))[:10])  # Ensure positive, consistent ID

        # Process the resume text
        processed_resume = preprocess_text(resume_text)
        summarized_resume = summarizer.summarize(processed_resume)
        resume_embedding = encoder.encode_text(summarized_resume).tolist()

        # Store in Qdrant
        from qdrant_client.http.models import PointStruct
        record = PointStruct(
            id=int(resume_id),  # Qdrant expects an integer ID
            vector=resume_embedding,
            payload={
                'processed_resume': processed_resume,
                'token_count': len(word_tokenize(processed_resume)),
                'summarized_resume': summarized_resume
            }
        )

        qdrant_interface.client.upsert(
            collection_name="resumes",
            points=[record]
        )

        return jsonify({
            'status': 'success',
            'resume_id': resume_id,
            'message': 'Resume added successfully'
        })

    except Exception as e:
        return jsonify({'error': f'Internal Server Error: {str(e)}'}), 500

# Start ngrok when the app starts
def start_ngrok():
    # Set up ngrok
    ngrok_auth_token = "2TVBcHqlSo65Dxpsbc8OF05sXcf_8afmhf2HXQ9QHzxjAYqu2"  # Replace with your ngrok auth token
    ngrok.set_auth_token(ngrok_auth_token)

    # Open an ngrok tunnel to the HTTP server
    public_url = ngrok.connect(5000).public_url
    print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000\"")

    # Update any base URLs to use the ngrok URL
    app.config["BASE_URL"] = public_url

# Run the Flask app
if __name__ == '__main__':
    import re  # Add this import at the top
    # Uncomment the next line to use ngrok
    start_ngrok()
    app.run(host='0.0.0.0', port=5000)


 * ngrok tunnel "https://b70d-34-148-79-83.ngrok-free.app" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
